In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install elasticsearch

     |████████████████████████████████| 225kB 2.0MB/s 


In [ ]:
# Pipeline for converting raw CICFlowmeter data into model friendly format

import os
import pandas as pd
import numpy as np
from joblib import dump, load
#import requests
from elasticsearch import Elasticsearch

df = pd.read_csv('/home/ai/Downloads/Anomaly_detection/ANOMALY_DETECTION_Project/ANOMALY_DETECTION_RUN/DATASET.csv')

#df = pd.read_csv('/content/drive/My Drive/AI project/AI Project/ANOMALY_DETECTION_RUN/DATASET.csv')

In [ ]:
# Reengineer ports
df['is_well_known_port'] = df.apply(lambda row: 1 if row['Dst Port'] < 1024 else 0, axis=1)
df['is_registered_port'] = df.apply(lambda row: 1 if row['Dst Port'] >= 1024 and row['Dst Port'] < 49152 else 0, axis=1)

# Drop certain columns

cols_to_drop = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
                'Timestamp', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg',
                'Fwd Bulk Rate Avg', 'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg', 'Label']

df_out = df.drop(columns=cols_to_drop, axis=1)

# Reorganize column order


# Scale columns


In [ ]:
df

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,is_well_known_port,is_registered_port
0,192.168.3.115-142.166.14.70-3703-80-6,192.168.3.115,3703,142.166.14.70,80,6,12/06/2010 08:31:08 AM,12325,1,1,...,0.0,0.0,0.0,1.276312e+15,0.0,1.276312e+15,1.276312e+15,NeedManualLabel,1,0
1,192.168.2.110-91.195.240.124-3905-80-6,192.168.2.110,3905,91.195.240.124,80,6,12/06/2010 08:31:08 AM,609184,2,0,...,0.0,0.0,0.0,1.276312e+15,0.0,1.276312e+15,1.276312e+15,NeedManualLabel,1,0
2,192.168.3.115-142.166.14.80-3704-80-6,192.168.3.115,3704,142.166.14.80,80,6,12/06/2010 08:31:08 AM,8625,1,1,...,0.0,0.0,0.0,1.276312e+15,0.0,1.276312e+15,1.276312e+15,NeedManualLabel,1,0
3,192.168.2.110-212.227.0.82-3914-80-6,192.168.2.110,3914,212.227.0.82,80,6,12/06/2010 08:31:06 AM,2515425,2,0,...,0.0,0.0,0.0,1.276312e+15,0.0,1.276312e+15,1.276312e+15,NeedManualLabel,1,0
4,192.168.2.110-212.227.116.171-3917-80-6,192.168.2.110,3917,212.227.116.171,80,6,12/06/2010 08:31:08 AM,1484216,2,0,...,0.0,0.0,0.0,1.276312e+15,0.0,1.276312e+15,1.276312e+15,NeedManualLabel,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243796,142.166.14.69-192.168.2.113-80-1387-6,142.166.14.69,80,192.168.2.113,1387,6,12/06/2010 01:20:00 PM,174,1,1,...,0.0,0.0,0.0,1.276329e+15,0.0,1.276329e+15,1.276329e+15,NeedManualLabel,0,1
243797,192.168.5.122-198.164.30.2-59564-53-17,192.168.5.122,59564,198.164.30.2,53,17,12/06/2010 01:03:39 PM,383414,1,1,...,0.0,0.0,0.0,1.276328e+15,0.0,1.276328e+15,1.276328e+15,NeedManualLabel,1,0
243798,192.168.5.124-192.168.5.122-44790-53-17,192.168.5.124,44790,192.168.5.122,53,17,13/06/2010 03:07:24 AM,499,1,1,...,0.0,0.0,0.0,1.276379e+15,0.0,1.276379e+15,1.276379e+15,NeedManualLabel,1,0
243799,192.168.5.122-198.164.30.2-43235-53-17,192.168.5.122,43235,198.164.30.2,53,17,12/06/2010 09:42:17 AM,16505,1,1,...,0.0,0.0,0.0,1.276316e+15,0.0,1.276316e+15,1.276316e+15,NeedManualLabel,1,0


In [ ]:
scaler = load('/home/ai/Downloads/Anomaly_detection/ANOMALY_DETECTION_Project/ANOMALY_DETECTION_Master/new_model/std_scaler_cicids17.bin')

cols = list(df_out.columns.values)
cols.remove('Fwd PSH Flags')
cols.remove('Fwd URG Flags')
cols.remove('FIN Flag Count')
cols.remove('SYN Flag Count')
cols.remove('RST Flag Count')
cols.remove('PSH Flag Count')
cols.remove('ACK Flag Count')
cols.remove('URG Flag Count')
cols.remove('CWR Flag Count')
cols.remove('ECE Flag Count')
cols.remove('is_well_known_port')
cols.remove('is_registered_port')

df_out[cols] = scaler.fit(df_out[cols]).transform(df_out[cols])


In [ ]:
model = load('/home/ai/Downloads/Anomaly_detection/ANOMALY_DETECTION_Project/ANOMALY_DETECTION_Master/new_model/rf_cicids2017.joblib') 

y_pred = model.predict(df_out)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.4s finished


In [ ]:
df['predicted'] = y_pred
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [ ]:
es = Elasticsearch("ec2-13-52-219-50.us-west-1.compute.amazonaws.com:9200")
#https://search-malmenator-hodt3wt2k7b5x7ph63zibt3eiy.us-east-1.es.amazonaws.com/

In [ ]:
es

<Elasticsearch([{'host': 'ec2-13-52-219-50.us-west-1.compute.amazonaws.com', 'port': 9200}])>

In [ ]:
vals = df.T.to_dict().values()

In [ ]:
for i in vals:
    es.index(index="anomaly_detection_new", doc_type="_doc", body=i)

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='ec2-13-52-219-50.us-west-1.compute.amazonaws.com', port=9200): Read timed out. (read timeout=10))

In [ ]:
print("Data sent to ES")